<a href=https://keras.io/examples/vision/conv_lstm/>LSTM</a><br>
<a href=https://stanford.edu/%7Eshervine/blog/keras-how-to-generate-data-on-the-fly>Dataset Class</a>

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from data import DataGenerator
import matplotlib.pyplot as plt
import os

2023-08-15 16:01:14.268497: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'data'

## Test Stardist

In [ ]:
from stardist.models import StarDist2D
from lamprogen.recipes.dl import stardist_2d_slicewise
import tifffile as tiff

modeldir = f'models'
name = 'lamprogen-stardist-trained'
model = StarDist2D(None, name=name,basedir=modeldir)

# im = tiff.imread('data/images/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif')
# labeled_3d = stardist_2d_slicewise(im, model)

In [ ]:
import napari

im = tiff.imread('data/inputs/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif')
mask = tiff.imread('data/masks/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif')
viewer=napari.Viewer()

viewer.add_image(im, name='intensity')
viewer.add_labels(mask, name='labels')

## Define Dataset

In [ ]:
import tifffile as tiff
image = tiff.imread("data/masks/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif")
image.shape

In [ ]:
def get_ids(path='data'):
    images = []
    masks = []
    for file in os.listdir(path+'/inputs'):
        images.append(file)

    for file in os.listdir(path+'/masks'):
        masks.append(file)

    return images, masks

images, masks = get_ids()
images.sort()
masks.sort()
images = images[:5]
masks = images[:5]

X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.1)

In [ ]:
print(X_train[3])
print(y_train[3])
print(X_test[0])
print(y_test[0])

In [ ]:
batch_size = 2

params = {'dim': (70,256,256),
          'batch_size': batch_size,
          'n_channels': 1,
          'shuffle': True}


training_generator = DataGenerator(X_train, y_train, **params)
validation_generator = DataGenerator(X_test, y_test, **params)

In [ ]:
print(training_generator[0][0].shape)
type(training_generator)

In [ ]:
# # Construct a figure on which we will visualize the images.
# fig, axes = plt.subplots(7, 10, figsize=(20, 16))

# # Plot each of the sequential images for one random data example.
# image = training_generator[0][0][0]
# for idx, ax in enumerate(axes.flat):
#     ax.imshow(np.squeeze(image[idx].astype('uint8')), cmap="magma")
#     ax.set_title(f"{idx + 1}")
#     ax.axis("off")

# # Print information and display the figure.
# print(f"Displaying frames for example {0}.")
# plt.show()

## Define Model

In [ ]:
#Load Packages
import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, ConvLSTM2D, BatchNormalization

model = Sequential()

model.add(ConvLSTM2D(filters=3, kernel_size=(3, 3),
                   input_shape=(70, 256, 256, 1),
                   padding='same', return_sequences=True, activation='relu', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True, activation='relu', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True, activation='relu', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True, activation='relu', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last', kernel_initializer='HeUniform'))

optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

## Define Traing Loop

In [ ]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 2

checkpoint_path = "LSTM Training"
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Fit the model to the training data.
model.fit(
    training_generator,
    batch_size = batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr, cp_callback],
)

In [ ]:
batch = []

im1 = tiff.imread('data/inputs/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif')
im1 = np.expand_dims(im1, axis=-1)
batch.append(im1)

im2 = tiff.imread('data/inputs/230413 GLA HEX grad post v2_Plate_1081_well_D05_channel2_0.tif')
im2 = np.expand_dims(im2, axis=-1)
batch.append(im2)
batch = np.asarray(batch)
print(batch.shape)

output = model.predict(batch)

In [ ]:
output

In [ ]:
X = np.empty((70,256,256,1))
X[:,:,:,:1]= im

test = X[None, :]
test.shape

In [ ]:
output = model.predict(test)

In [ ]:
output.shape

prediction = np.squeeze(output)
prediction.shape

In [ ]:
viewer=napari.Viewer()

im = tiff.imread('data/inputs/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif')
viewer.add_image(im, name='intensity')
viewer.add_labels(prediction.astype("uint8"), name='prediction')

In [ ]:
output